# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the data.
2. Build a feedforward neural network to classify traffic signs.
3. Build a convolutional neural network to classify traffic signs.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

## Load the Data

Start by importing the data from the pickle file.

In [1]:
import pickle


In [2]:
# TODO: Implement load the data here.
training_file = 'train.p'
testing_file = 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

    
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

## Normalize the data

Now that you've loaded the training data, normalize the input so that it has a mean of 0 and a range between -0.5 and 0.5.

In [3]:
import numpy as np
def normalize(data):
    a = -0.5
    b = 0.5
    
    data = data - data.mean(axis=0)
    
    xmin = np.min(data)
    ymin = np.max(data)
        
#     data = data / np.max(data)
    result  = a + ((data - xmin) / (ymin-xmin)) * (b-a) 

    return result

In [4]:
# TODO: Implement data normalization here.
from sklearn import preprocessing
X_train = normalize(X_train)
X_test = normalize(X_test)

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(round(np.mean(X_train)) == 0), "The mean of the input data is: %f" % np.mean(X_train)
assert(np.min(X_train) == -0.5 and np.max(X_train) == 0.5), "The range of the input data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Activation


Using TensorFlow backend.


In [6]:
#model for the quiz
model = Sequential()
model.add(Dense(128, input_dim=3072,activation="relu",name="hidden1"))
model.add(Dense(43,activation="softmax",name="output"))


In [7]:
print(model.get_layer(name="hidden1"))

In [8]:
# TODO: Build a two-layer feedforward neural network with Keras here.


# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(model.get_layer(name="hidden1").input_shape == (None, 32*32*3)), "The input shape is: %s" % model.get_layer(name="hidden1").input_shape
assert(model.get_layer(name="output").output_shape == (None, 43)), "The output shape is: %s" % model.get_layer(name="output").output_shape 

## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [9]:
import keras
K_adam_opt = keras.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=K_adam_opt, loss="categorical_crossentropy",metrics=["accuracy"])

In [10]:
#reshape the input 
print(X_train.shape)
input_size = 32*32*3
X_train_flat = X_train.reshape((-1,32*32*3))
print(X_train_flat.shape)
X_test_flat = X_test.reshape((-1,32*32*3))

(39209, 32, 32, 3)
(39209, 3072)


In [11]:
#convert to one hot
from keras.utils.np_utils import to_categorical
y_train_onehot = to_categorical(y_train,43)
y_test_onehot = to_categorical(y_test,43)
print(y_train_onehot[1])

[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.]


In [12]:
# TODO: Compile and train the model here.
history = model.fit(X_train_flat, y_train_onehot, nb_epoch=2, batch_size=64 )

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][0] > 0.5), "The training accuracy was: %.3f" % history.history['acc'][0]

Epoch 1/2
39209/39209 [==============================] - 2s - loss: 1.7956 - acc: 0.5420     
Epoch 2/2
39209/39209 [==============================] - 2s - loss: 0.8760 - acc: 0.7787     


## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [13]:
# TODO: Split some of the training data into a validation dataset.
# TODO: Compile and train the model to measure validation accuracy.
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train_flat, y_train_onehot, test_size=0.25, random_state=24)


In [14]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)


(29406, 3072)
(29406, 43)
(9803, 3072)


In [15]:

history = model.fit(X_train,y_train, nb_epoch=2, batch_size=10000 ,validation_data=(X_val,y_val))
print(history.history['val_acc'][0])
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(round(X_train.shape[0] / float(X_val.shape[0])) == 3), "The training set is %.3f times larger than the validation set." % (X_train.shape[0] / float(X_val.shape[0]))
assert(history.history['val_acc'][0] > 0.6), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 0s - loss: 0.7193 - acc: 0.8276 - val_loss: 0.7046 - val_acc: 0.8432
Epoch 2/2
29406/29406 [==============================] - 0s - loss: 0.6803 - acc: 0.8455 - val_loss: 0.6737 - val_acc: 0.8481
0.84321141243


**Validation Accuracy**: 0.82750

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [16]:
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten


# input image dimensions
img_rows, img_cols, img_channels = 32, 32,3
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

input_shape = input_shape = (img_rows, img_cols, img_channels)

#reshape the data
X_train = X_train.reshape(29406,32,32,3)
X_train = X_train.astype('float32')
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, img_channels)
X_test = X_test.astype('float32')
print(X_train.shape)
print(X_test.shape)
model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Flatten())
model.add(Dense(128, input_dim=3072,activation="relu",name="hidden1"))
model.add(Dense(43,activation="softmax",name="output"))


(29406, 32, 32, 3)
(12630, 32, 32, 3)


In [17]:
## TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
# TODO: Compile and train the model.
K_adam_opt = keras.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=K_adam_opt, loss="categorical_crossentropy",metrics=["accuracy"])
history = model.fit(X_train,y_train, nb_epoch=2, batch_size=64 ,validation_data=(X_val,y_val))

print(history.history['val_acc'])
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 5s - loss: 0.9855 - acc: 0.7475 - val_loss: 0.3902 - val_acc: 0.9026
Epoch 2/2
29406/29406 [==============================] - 5s - loss: 0.2445 - acc: 0.9419 - val_loss: 0.2411 - val_acc: 0.9423
[0.90258084259920435, 0.94226257268183211]


**Validation Accuracy**: (fill in here)

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [18]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.
# size of pooling area for max pooling
pool_size = (2, 2)


model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Flatten())
model.add(Dense(128, input_dim=3072,activation="relu",name="hidden1"))
model.add(Dense(43,activation="softmax",name="output"))

In [19]:
# TODO: Compile and train the model.
model.compile(optimizer=K_adam_opt, loss="categorical_crossentropy",metrics=["accuracy"])
history = model.fit(X_train,y_train, nb_epoch=2, batch_size=64 ,validation_data=(X_val,y_val))
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 3s - loss: 0.8696 - acc: 0.7722 - val_loss: 0.3354 - val_acc: 0.9142
Epoch 2/2
29406/29406 [==============================] - 3s - loss: 0.2242 - acc: 0.9498 - val_loss: 0.2076 - val_acc: 0.9551


**Validation Accuracy**: (fill in here)

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [20]:
# TODO: Re-construct the network and add dropout after the pooling layer.

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, input_dim=3072,activation="relu",name="hidden1"))
model.add(Dense(43,activation="softmax",name="output"))

In [21]:
# TODO: Compile and train the model.
model.compile(optimizer=K_adam_opt, loss="categorical_crossentropy",metrics=["accuracy"])
history = model.fit(X_train,y_train, nb_epoch=2, batch_size=64 ,validation_data=(X_val,y_val))
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 4s - loss: 0.9420 - acc: 0.7515 - val_loss: 0.3716 - val_acc: 0.9202
Epoch 2/2
29406/29406 [==============================] - 3s - loss: 0.2898 - acc: 0.9282 - val_loss: 0.2283 - val_acc: 0.9548


**Validation Accuracy**: 0.9478

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

In [22]:
model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, input_dim=3072,activation="relu",name="hidden1"))
model.add(Dropout(0.5))
model.add(Dense(43,activation="softmax",name="output"))

model.compile(optimizer=K_adam_opt, loss="categorical_crossentropy",metrics=["accuracy"])
history = model.fit(X_train,y_train, nb_epoch=50, batch_size=64 ,validation_data=(X_val,y_val))
print(history.history['val_acc'])

Train on 29406 samples, validate on 9803 samples
Epoch 1/50
29406/29406 [==============================] - 4s - loss: 1.7101 - acc: 0.5115 - val_loss: 0.6173 - val_acc: 0.8568
Epoch 2/50
29406/29406 [==============================] - 4s - loss: 0.7475 - acc: 0.7730 - val_loss: 0.3570 - val_acc: 0.9150
Epoch 3/50
29406/29406 [==============================] - 4s - loss: 0.5635 - acc: 0.8278 - val_loss: 0.2605 - val_acc: 0.9466
Epoch 4/50
29406/29406 [==============================] - 4s - loss: 0.4784 - acc: 0.8537 - val_loss: 0.2221 - val_acc: 0.9555
Epoch 5/50
29406/29406 [==============================] - 4s - loss: 0.4156 - acc: 0.8741 - val_loss: 0.1825 - val_acc: 0.9610
Epoch 6/50
29406/29406 [==============================] - 4s - loss: 0.3705 - acc: 0.8877 - val_loss: 0.1535 - val_acc: 0.9669
Epoch 7/50
29406/29406 [==============================] - 4s - loss: 0.3369 - acc: 0.8970 - val_loss: 0.1465 - val_acc: 0.9706
Epoch 8/50
29406/29406 [==============================] - 4s -

**Best Validation Accuracy:** (fill in here)

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: After you load your test data, don't forget to normalize the input and one-hot encode the output, so it matches the training data.

Hint 2: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [23]:
with open('./test.p', mode='rb') as f:
    test = pickle.load(f)
    
X_test = test['features']
y_test = test['labels']
X_test = X_test.astype('float32')
X_test /= 255
X_test -= 0.5
Y_test = to_categorical(y_test, 43)

score = model.evaluate(X_test, Y_test)
print('Test score:', score[0])
print('Test accuracy:', score[1])

12630/12630 [==============================] - 1s     
Test score: 0.294277059278
Test accuracy: 0.928424386401


**Test Accuracy:** 92.48%


## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.